In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 
from hnmchallenge.data_reader import DataReader
import datetime


In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
dataset = FilterdDataset()
dr = DataReader()

In [4]:
train = dataset.get_train_df()


In [5]:
train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2
...,...,...,...,...,...
14094719,2020-09-14,422963,15729,0.025407,1
14094720,2020-09-14,422963,119,0.033881,1
14094721,2020-09-14,422963,14206,0.030492,1
14094722,2020-09-14,422963,8827,0.016932,1


In [6]:
train["tdiff"]=train['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,23) - x).days)

In [8]:
train

,t_dat,customer_id,article_id,price,sales_channel_id,tdiff
0,2018-09-20,0,0,0.008458,2,0.001362
1,2018-09-20,0,0,0.008458,2,0.001362
2,2018-09-20,1,1,0.033881,1,0.001362
3,2018-09-20,2,2,0.021593,1,0.001362
4,2018-09-20,3,3,0.022017,2,0.001362
...,...,...,...,...,...,...
14094719,2020-09-14,422963,15729,0.025407,1,0.111111
14094720,2020-09-14,422963,119,0.033881,1,0.111111
14094721,2020-09-14,422963,14206,0.030492,1,0.111111
14094722,2020-09-14,422963,8827,0.016932,1,0.111111


In [9]:
train["max_price"] = train.groupby(DEFAULT_ITEM_COL)["price"].transform("max")
train["sale_factor"] = (1 - (train["price"] / train["max_price"]))


In [10]:
train["last_buy"] = train.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)
train["first_buy"] = train.groupby(DEFAULT_USER_COL)["t_dat"].transform(min)
train["time_score"] = ((train["t_dat"] - train["first_buy"])/ (train["last_buy"] - train["first_buy"])) 
train.drop(["last_buy", "first_buy"], axis=1)

In [12]:
train

,t_dat,customer_id,article_id,price,sales_channel_id,tdiff,max_price,sale_factor,time_score
0,2018-09-20,0,0,0.008458,2,0.001362,0.008458,0.0000,0.0
1,2018-09-20,0,0,0.008458,2,0.001362,0.008458,0.0000,0.0
2,2018-09-20,1,1,0.033881,1,0.001362,0.033881,0.0000,0.0
3,2018-09-20,2,2,0.021593,1,0.001362,0.025407,0.1501,0.0
4,2018-09-20,3,3,0.022017,2,0.001362,0.022017,0.0000,0.0
...,...,...,...,...,...,...,...,...,...
14094719,2020-09-14,422963,15729,0.025407,1,0.111111,0.025407,0.0000,1.0
14094720,2020-09-14,422963,119,0.033881,1,0.111111,0.033881,0.0000,1.0
14094721,2020-09-14,422963,14206,0.030492,1,0.111111,0.030492,0.0000,1.0
14094722,2020-09-14,422963,8827,0.016932,1,0.111111,0.016932,0.0000,1.0


In [13]:

train1 = dataset.get_train_df()
item_per_user1 = train1.groupby("customer_id")["article_id"].apply(list)
unique_item_per_user1 = item_per_user1.apply(np.unique)
df_multiple1 = item_per_user1.to_frame()
df_unique1 = unique_item_per_user1.to_frame()
df_multiple1["count"] = df_multiple1.apply(
    lambda row: len(row["article_id"]), axis=1
)
df_unique1["count"] = df_unique1.apply(lambda row: len(row["article_id"]), axis=1)
merge_df1 = pd.merge(df_multiple1, df_unique1, on="customer_id")
merge_df1["diff"] = 1 - (merge_df1["count_y"] / merge_df1["count_x"])
user_diff = merge_df1.drop(
    ["article_id_x", "count_x", "article_id_y", "count_y"], axis=1
)
user_diff = user_diff.reset_index()

In [14]:
user_diff

,customer_id,diff
0,0,0.214286
1,1,0.125000
2,2,0.000000
3,3,0.428571
4,4,0.000000
...,...,...
1128711,1128711,0.000000
1128712,1128712,0.000000
1128713,1128713,0.000000
1128714,1128714,0.000000


z=train.join(user_diff.set_index('customer_id'), on='customer_id')

In [15]:
z=train.join(user_diff.set_index('customer_id'), on='customer_id')

In [17]:
z

,t_dat,customer_id,article_id,price,sales_channel_id,tdiff,max_price,sale_factor,time_score,diff
0,2018-09-20,0,0,0.008458,2,0.001362,0.008458,0.0000,0.0,0.214286
1,2018-09-20,0,0,0.008458,2,0.001362,0.008458,0.0000,0.0,0.214286
2,2018-09-20,1,1,0.033881,1,0.001362,0.033881,0.0000,0.0,0.125000
3,2018-09-20,2,2,0.021593,1,0.001362,0.025407,0.1501,0.0,0.000000
4,2018-09-20,3,3,0.022017,2,0.001362,0.022017,0.0000,0.0,0.428571
...,...,...,...,...,...,...,...,...,...,...
14094719,2020-09-14,422963,15729,0.025407,1,0.111111,0.025407,0.0000,1.0,0.000000
14094720,2020-09-14,422963,119,0.033881,1,0.111111,0.033881,0.0000,1.0,0.000000
14094721,2020-09-14,422963,14206,0.030492,1,0.111111,0.030492,0.0000,1.0,0.000000
14094722,2020-09-14,422963,8827,0.016932,1,0.111111,0.016932,0.0000,1.0,0.000000


In [18]:
fd = dataset.get_train_df()
duplicated_rows = fd[fd.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])]
count_mb = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
count_mb = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
    columns={"t_dat": "count"}
)

In [19]:
count_mb

,article_id,count
0,0,633
1,1,335
2,2,168
3,3,457
4,4,2328
...,...,...
19129,21252,1
19130,21253,1
19131,21271,2
19132,21274,1


In [23]:
z1=z.join(count_mb.set_index('article_id'), on='article_id')

In [24]:
z1['count']=z1['count'].fillna(0)

In [25]:
z1

,t_dat,customer_id,article_id,price,sales_channel_id,tdiff,max_price,sale_factor,time_score,diff,count
0,2018-09-20,0,0,0.008458,2,0.001362,0.008458,0.0000,0.0,0.214286,633.0
1,2018-09-20,0,0,0.008458,2,0.001362,0.008458,0.0000,0.0,0.214286,633.0
2,2018-09-20,1,1,0.033881,1,0.001362,0.033881,0.0000,0.0,0.125000,335.0
3,2018-09-20,2,2,0.021593,1,0.001362,0.025407,0.1501,0.0,0.000000,168.0
4,2018-09-20,3,3,0.022017,2,0.001362,0.022017,0.0000,0.0,0.428571,457.0
...,...,...,...,...,...,...,...,...,...,...,...
14094719,2020-09-14,422963,15729,0.025407,1,0.111111,0.025407,0.0000,1.0,0.000000,65.0
14094720,2020-09-14,422963,119,0.033881,1,0.111111,0.033881,0.0000,1.0,0.000000,1767.0
14094721,2020-09-14,422963,14206,0.030492,1,0.111111,0.030492,0.0000,1.0,0.000000,7.0
14094722,2020-09-14,422963,8827,0.016932,1,0.111111,0.016932,0.0000,1.0,0.000000,134.0


In [26]:
dr = DataReader()
z1.to_feather(
        dr.get_preprocessed_data_path() / "filtered_feature_dataset.feather"
    )

PermissionError: [Errno 13] Permission denied: '/home/aayush/../edoardo/hnm_data/preprocessed/filtered_feature_dataset.feather'